In [1]:
from ortools.init import pywrapinit
from numpy import linalg as LA
from math import radians, cos, sin, asin, sqrt, dist
from scipy.stats import truncnorm
from ortools.sat.python import cp_model

import numpy as np
import pandas as pd
import geopy.distance
import rvo2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import random


In [2]:
#---------------------------------- EXTRACT THE DATA ----------------------------------

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
users_path = '../eua-dataset/users/'
U = pd.read_csv(users_path + 'users-test.csv')

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
nodes_path = '../eua-dataset/edge-servers/'
N_src = pd.read_csv(nodes_path + 'serverstest.csv')
N_dest = pd.read_csv(nodes_path + 'serverstest.csv')

In [3]:
#-----------------------------------INPUTS-----------------------------------

#Incoming requests to node i
#Function(rows) x Node(column) [request/sec]
lambda_ri=[[1,0,0],[1,0,1],[1,1,1],[2,0,0]] 

#Amount of request received in time-slot
R=np.sum(lambda_ri)

#Identifies which user sent the request [U x R]
req_u =np.zeros([len(U),R])
req_u=[[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]]

#Ammount of arrivals per node
ammount_arrivals_n = []
for i in range(len(N_src)):
    ammount_arrivals_n.append(np.sum(np.transpose(lambda_ri)[i]))

 # Memory of function: m_f
m1 = 1
m2 = 2
m3 = 3
m4 = 4
m_f = [m1,m2,m3,m4]

#Maximum allowed network delay for function f:  phi_f
phi_f=[]

In [4]:
#-----------------------------------CRITICALITY INPUTS-----------------------------------

#D = (-37.81952,144.95714099999998) # Danger source position
D = (-37.8183519,144.9570042)
D_rad = (0.1)#/111.139 # Influence range of danger source D (radius) in km


T_1 =0 # Starting point of period
T_2 = 1 # Ending point of period
T = T_2-T_1 # Total interval time

U_per = np.full(len(U),0.2) # Perception range of individual uj in km
lambd = 0.5 # Severity of the stimulus event
#nej = np.random.uniform(0,1,len(U)) # Emotional fluctuation of uj ---> nej ∈ (0, 1)
nej = np.full(len(U),0.5)
#se_j =np.random.uniform(0.05,0.1, len(U)) # Individual sensitivity uj 
se_j = np.full(len(U),0.1)

#Saves the coordinates from which user the request was sent from
positions = []

for r in range(R):
    temp=[]
    for u in range(len(U)):
        if req_u[u][r]==1:
            temp=[[U.iloc[u]['Latitude'],U.iloc[u]['Longitude']]]
    positions.append(temp)

In [5]:
#---------------------------------- CRITICALITY MODEL ----------------------------------

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def criticality (time,du_Dt):
    # emj: emotional value
    em_t = np.zeros(len(U))
    for j in range(len(U)):
        if du_Dt[j] < (D_rad+U_per[j]):
            eq = ((time-T_1)/T) * (1-(du_Dt[j]/(D_rad+U_per[j])))*lambd* nej[j]
            em_t[j]=eq
        else:
            em_t[j]=0
    
    print(em_t)

    # scrj(t) = sej + emj (t) (subjective criticality equation)
    scr=np.zeros(len(U))
    for j in range(len(U)):
        scr[j]= (se_j[j]+em_t[j])

    ocr_j=np.zeros(len(U)) # ocr objective criticality of individual uj at time step t
    nDiv = 5 # Number of partitions of the coverage area
    circles = D_rad/nDiv # Radius of each circular partition
    covCircles = [] # Distance from the center
    ri = [0.5,0.4,0.3,0.2,0.1]

    for p in range(1,nDiv+1):
        covCircles.append(circles*p)

    for j in range(len(U)):
            if du_Dt[j]<covCircles[0]:
                ocr_j[j] = ri[0]

            elif du_Dt[j]<covCircles[1] and du_Dt[j]>=covCircles[0]:
                ocr_j[j]  =ri[1]

            elif du_Dt[j]<covCircles[2] and du_Dt[j]>=covCircles[1]:
                ocr_j[j]  = ri[2]

            elif du_Dt[j]<covCircles[3] and du_Dt[j]>=covCircles[2]:
                ocr_j[j] = ri[3]
            elif du_Dt[j]<covCircles[4] and du_Dt[j]>=covCircles[3]:
                ocr_j[j]  = ri[4]

            else:
                ocr_j[j]  = 0
    print(ocr_j)
    # Criticality equation
    # CRj = (L1 * scrj(t)) + (L2 * ocrj)
    mu=1
    sigma=0.05
    low=0.95
    up =1.05

    X = get_truncated_normal(mu, sigma, low, up)

    #L1 =X.rvs() # weight lambda 1
    L1 = 0.95
    L2 = 1 # weight lambda 2

    # Criticality
    CR = []

    for j in range(len(U)):
        cr = (L1*scr[j])+(L2*ocr_j[j]) 
        CR.append(cr)

    index_CR=sorted(range(len(CR)), key=lambda a: CR[a],reverse=True)
    
    return index_CR, CR

In [6]:
#---------------------------------- Update position users ----------------------------------
def du_dt_function(timeStep,positions):
    du_Dt=[] # Distance between danger and user
    for j in range(len(U)):
        user_latitude=positions[j][timeStep][0]
        user_longitude=positions[j][timeStep][1]
        user_coordinates = (user_latitude,user_longitude)
        dist_geoDanger = geopy.distance.geodesic(user_coordinates, D).km
        du_Dt.append(dist_geoDanger)
    return du_Dt

In [7]:
#--------------------------------Obtain criticality--------------------------------------
#for t in range(T):
ti=1
du_dt_temp=du_dt_function(ti-1,positions)
index_CR_t,CR_t =criticality(ti,du_dt_temp)

#show which requests are assigned to each function [F x R]
req_dist = np.zeros([len(m_f),R])

crit_range = 1/len(m_f)
f_cr =np.zeros([len(m_f),2])
#f_cr=[[0.4,0.42],[0.42,0.44],[0.44,0.46],[0.46,0.48]]
temp=0
for f in range(len(m_f)):
    f_cr[f][0]=temp
    f_cr[f][1]=temp+crit_range
    temp=temp+crit_range

for f in range(len(m_f)):
    for r in range(R):
        if CR_t[r]>=f_cr[f][0]:
            if CR_t[r]<f_cr[f][1]:
                req_dist[f][r]=1

[0.24630765 0.21801947 0.24167999 0.20598881 0.21682093 0.23517596
 0.22562908 0.24308797]
[0.5 0.4 0.5 0.3 0.4 0.5 0.4 0.5]


In [8]:
#-----------------------------------INFRASTRUCTURED DATA-----------------------------------
#Memory available in node j: M_j
M_j = [10,10,10]

#Cores on node j: U_j
U_j=[16,16,16]

In [9]:
#-----------------------------------MONITORED DATA-----------------------------------
#Network delay between nodes i and j
delta_ij = []

#Cores used by node j per single request: u_rj 
u_rj = [2,2,2,2,2,2,2,2]

In [10]:
#---------------------------------- VARIABLES ----------------------------------
U_ni = [] # Set of requests allocated to node i

In [11]:
#-----------------------------------HAVERSINE-----------------------------------

def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [12]:
#-----------------------------------COVERAGE REQUEST-NODE-----------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node_latitude = N_src.iloc[i]['LATITUDE']
  node_longitude = N_src.iloc[i]['LONGITUDE']
  temp = []
  t_distance =[]
  for r in range(R):
    for u in range(len(U)):

      if req_u[u][r]==1:
        request_latitude = U.iloc[u]['Latitude']
        request_longitude = U.iloc[u]['Longitude']

        dist_geo = haversine(node_longitude, node_latitude, request_longitude, request_latitude)
        t_distance.append(np.round(dist_geo,3))
        if dist_geo <= radius[i]:
          temp.append(1)
        
        else:
          temp.append(0)
       
  U_ni.append(temp)


In [13]:
#---------------------------------DISTANCE BETWEEN NODES---------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node1_latitude = N_src.iloc[i]['LATITUDE']
  node1_longitude = N_src.iloc[i]['LONGITUDE']
  temp_dist = []
  for j in range(len(N_dest)):
    node2_latitude = N_dest.iloc[j]['LATITUDE']
    node2_longitude = N_dest.iloc[j]['LONGITUDE']

    dist_geo_nodes = haversine(node1_longitude, node1_latitude, node2_longitude,node2_latitude)
    #print(dist_geo)
    #print(radius[i]+radius[j])
    temp_dist.append(np.round(dist_geo_nodes,3))
       
  delta_ij.append(temp_dist)

for r1 in range(len(N_src)):
  temp_delay = []
  for r2 in range(len(N_dest)):
    if r1==r2:
      temp_delay.append(0)
    else:
      temp_delay.append(radius[r1]+radius[r2])
  phi_f.append(temp_delay)
    

In [14]:
#---------------------------------- CP MODEL ----------------------------------
model = cp_model.CpModel()

#-----------------------------------SOLVER VARIABLES-----------------------------------

# x_j,r = True if request r is allocated to node j
x = {}
for j in range(len(N_dest)):
    for r in index_CR_t:
        x[j, r] = model.NewBoolVar(f'c[{j}][{r}]')

# c_f,j = True if container f (fa,fb,fc) is deployed on node j
c = {}
for f in range(len(lambda_ri)):
    for j in range(len(N_dest)):
        c[f, j] = model.NewBoolVar(f'c[{f}][{j}]')

# y_j = True if node j is used
# y_j = False otherwise
y = {}
for j in range(len(N_dest)): 
    y[j] = model.NewBoolVar(f'c[{j}]')
 
print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]



In [15]:
#-----------------------------------CONSTRAINTS-----------------------------------
#Controls if request r can be managed by node j
for r in index_CR_t:
    for j in range(len(N_dest)):
        if U_ni[j][r]==0:
            model.Add(x[j, r]==0)

#Proximity constraint (node i-node j) 
for i in range(len(N_src)):
    for r in index_CR_t:
        for j in range(len(N_dest)):
                if delta_ij[i][j]> phi_f[i][j]:
                    model.Add(
                    x[j, r]==0
                    )  

#Container deployment: True if container of function f is deployed on node j
for f in range(len(lambda_ri)):
    for j in range(len(N_dest)):
        model.Add(sum([x[j,r] for r in index_CR_t])<= c[f,j]*1000)

# Capacity constraint (memory)
for j in range(len(N_dest)):
    model.Add(
                sum([
                     m_f[f]* c[f,j] for f in range(len(lambda_ri))
                    ]) <= M_j[j]*y[j]
                )

# Avoid resource contention
for j in range(len(N_dest)):
        model.Add(
                sum([sum([x[j,r]*lambda_ri[f][i]*u_rj[r] for r in index_CR_t]) for i in range(len(N_src))
                ]) <= U_j[j]*y[j]   
            ) 
        
# Contraint family (each request can be allocated just once)
for r in index_CR_t:
    model.Add(sum([x[j, r] for j in range(len(N_dest))]) <= 1)

 
print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]
#kLinear1: 11
#kLinear3: 8
#kLinearN: 18 (#terms: 150)


In [16]:
#---------------------------------- CP SOLVER ----------------------------------
solver = cp_model.CpSolver()

#---------------------------------- OBJECTIVE FUNCTIONS ---------------------------------

# Maximize the number of allocated requests
objective_max = []

for j in range(len(N_dest)):
    for r in index_CR_t:
        objective_max.append(x[j, r])
        #objective_max.append(cp_model._WeightedSum({x[j, r]},{CR_t[j]}))
model.Maximize(sum(objective_max))

solver.Solve(model)
max_requests = solver.ObjectiveValue()

# Hint (speed up solving)
for j in range(len(N_dest)):
     for r in index_CR_t:
        model.AddHint(x[j,r], solver.Value(x[j,r]))
    
# Constraint previous objective
model.Add(
    sum([
        x[j, r] for j in range(len(N_dest)) for r in index_CR_t
    ]) == round(solver.ObjectiveValue())
) 

# Minimize the number of nodes used
objective_min = []

for j in range(len(N_dest)):
    objective_min.append(y[j])
model.Minimize(sum(objective_min))

print(model.ModelStats())

optimization model '':
#Variables: 39 (#bools:3 in objective)
  - 39 Booleans in [0,1]
#kLinear1: 11
#kLinear3: 8
#kLinearN: 19 (#terms: 174)


In [17]:
#-----------------------------------CALL THE SOLVER-----------------------------------
status = solver.Solve(model)
x_rj = np.zeros([len(N_dest),R])
#-----------------------------------DISPLAY THE SOLUTION-----------------------------------

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('SOLUTION:')
    print(f'Objective value: {max_requests} requests have been allocated to {solver.ObjectiveValue()} nodes\n')
    for r in index_CR_t:
        for j in range(len(N_dest)):
            #print(f'r: {r} j: {j} solver: {solver.Value(x[j,r])}')
            if int(solver.Value(x[j,r])) == 1:
                print(f'x[{j},{r}]: Request {r} has been allocated to node {j}')
                x_rj[j][r]=1
                
    print('----------------------------------------------------------------------')
    for f in range(len(lambda_ri)):
        for j in range(len(N_dest)):
            if int(solver.Value(c[f,j])) == 1:
                print(f'c[{f},{j}]: Function {f} has been deployed on node {j}')

    print('----------------------------------------------------------------------')
    for j in range(len(N_dest)):
        if int(solver.Value(y[j])) == 1:
                print(f'y[{j}]: Node {j} is used') 
    #---------------------------------------------CERO VARIABLES----------------------------------------------------
    print('  ')
    print('----------------------------------------------------------------------')
    print('VARIABLES IN ZERO')
    print('----------------------------------------------------------------------')
    for r in index_CR_t:
        for j in range(len(N_dest)):
            if int(solver.Value(x[j,r])) == 0:
                print(f'x[{j},{r}]')
    print('----------------------------------------------------------------------')
    for f in range(len(lambda_ri)):
        for j in range(len(N_dest)):
            if int(solver.Value(c[f,j])) == 0:
                print(f'c[{f},{j}]')

    print('----------------------------------------------------------------------')
    for j in range(len(N_dest)):
        if int(solver.Value(y[j])) == 0:
                print(f'y[{j}]')
else:
    print('The problem does not have an optimal solution.')

#print("\n--- Run time: %s seconds ---" % round((time.time() - start_time),2))

SOLUTION:
Objective value: 8.0 requests have been allocated to 2.0 nodes

x[2,0]: Request 0 has been allocated to node 2
x[2,7]: Request 7 has been allocated to node 2
x[1,2]: Request 2 has been allocated to node 1
x[2,5]: Request 5 has been allocated to node 2
x[2,6]: Request 6 has been allocated to node 2
x[1,1]: Request 1 has been allocated to node 1
x[1,4]: Request 4 has been allocated to node 1
x[1,3]: Request 3 has been allocated to node 1
----------------------------------------------------------------------
c[0,1]: Function 0 has been deployed on node 1
c[0,2]: Function 0 has been deployed on node 2
c[1,1]: Function 1 has been deployed on node 1
c[1,2]: Function 1 has been deployed on node 2
c[2,1]: Function 2 has been deployed on node 1
c[2,2]: Function 2 has been deployed on node 2
c[3,1]: Function 3 has been deployed on node 1
c[3,2]: Function 3 has been deployed on node 2
----------------------------------------------------------------------
y[1]: Node 1 is used
y[2]: Node 

In [18]:
#-----------------------------------Normalization-----------------------------------
mat_mul=np.dot(req_dist,np.transpose(x_rj))
x_fj=np.zeros([len(lambda_ri),len(N_dest)])
for j in range(len(N_dest)):
    for f in range(len(lambda_ri)):
        if sum(mat_mul[f])==0:
            x_fj[f][j]=0
        else:
            x_fj[f][j]=mat_mul[f][j]/sum(mat_mul[f])
